# SUPER PIANO (XLNet Piano)

<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/SuperPiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Huge thanks goes to Andrew Shaw (bearpelican) for the MusicAutoBot repo/code that was used to create this notebook. This notebook is a fork of Andrew's notebook for MusicTransformer

https://github.com/bearpelican

https://github.com/bearpelican/musicautobot

# SETUP THE ENVIRONMENT

In [ ]:
#@title
!git clone https://github.com/asigalov61/musicautobot.git
import os
os.chdir('musicautobot')
!nvidia-smi
!apt install fluidsynth
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/musicautobot
!pip install torch fastai music21 pebble fluidsynth midi2audio
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.numpy_encode import stream2npenc_parts
from musicautobot.utils.midifile import *
from musicautobot.utils.setup_musescore import setup_musescore
from fastai.text import *
setup_musescore()
from midi2audio import FluidSynth
from IPython.display import Audio
# Colab cannot play music directly from music21 - must convert to .wav first
def play_wav(stream):
    out_midi = stream.write('midi')
    out_wav = str(Path(out_midi).with_suffix('.wav'))
    FluidSynth("/content/musicautobot/Yamaha-C5-Salamander-JNv5.1.sf2").midi_to_audio(out_midi, out_wav)
    return Audio(out_wav)
from google.colab import files


Mount your Google Drive (Highly recommended)

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

(OPTIONAL) Download SuperPiano original and ready-to-use training data and model

In [ ]:
#@title
!wget 'https://superpiano.s3-us-west-1.amazonaws.com/training-data-piano-no-dynamics.pkl'
!wget 'https://superpiano.s3-us-west-1.amazonaws.com/SuperPianoModel.pth'

# PREP TRAINING DATA

Initialize the paths and variables

In [ ]:
# Location of your midi filesfiles
midi_path = Path('/content/musicautobot')
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path('/content/musicautobot')
data_path.mkdir(parents=True, exist_ok=True)

# Location of preprocessed numpy files
numpy_path = Path('/content/musicautobot/lm')

# Location to save trained model/take saved model checkpoint
model_path = Path('/content/musicautobot')

data_save_name = 'training-data-piano-no-dynamics.pkl'
[p.mkdir(parents=True, exist_ok=True) for p in [midi_path, numpy_path, data_path, model_path]];

Upload your own MIDI files

In [ ]:
from google.colab import files
uploaded = files.upload()

Download and parse Google's Magenta MAESTRO Dataset

In [ ]:
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j /content/musicautobot/maestro-v2.0.0-midi.zip

MIDI PROCESSING

Calculate and display number of indexed MIDI files

In [ ]:
# num_tracks = [1, 2] # number of tracks to support
cutoff = 5 # max instruments
min_variation = 3 # minimum number of different midi notes played
# max_dur = 128

midi_files = get_files(midi_path, '.mid', recurse=False); len(midi_files)

Parse, Compress, and Convert MIDI files into Numpy format

In [ ]:
def process_metadata(midi_file):
    # Get outfile and check if it exists
    out_file = numpy_path/midi_file.relative_to(midi_path).with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return
    
    npenc = transform_midi(midi_file)
    if npenc is not None: np.save(out_file, npenc)

#===============================================================

def transform_midi(midi_file):
    input_path = midi_file
    
    # Part 1: Filter out midi tracks (drums, repetitive instruments, etc.)
    try: 
#         if duet_only and num_piano_tracks(input_path) not in [1, 2]: return None
        input_file = compress_midi_file(input_path, min_variation=min_variation, cutoff=cutoff) # remove non note tracks and standardize instruments
        
        if input_file is None: return None
    except Exception as e:
        if 'badly form' in str(e): return None # ignore badly formatted midi errors
        if 'out of range' in str(e): return None # ignore badly formatted midi errors
        print('Error parsing midi', input_path, e)
        return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        print('Could not encode to chordarr:', input_path, e)
        print(traceback.format_exc())
        return None
    
    # Part 3. Compress song rests - Don't want songs with really long pauses 
    # (this happens because we filter out midi tracks).
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
    if delta_trim > 500: 
         print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
         return None
    chordarr = chord_short
    
    # Part 3. Chord array to numpy
    npenc = chordarr2npenc(chordarr)
    if not is_valid_npenc(npenc, input_path=input_path):
        return None
    
    return npenc

#===============================================================
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

#===============================================================
processed = process_all(process_metadata, midi_files, timeout=120, timeout_func=timeout_func)

Calculate and display number of resulting Numpy files

In [ ]:
def create_databunch(files, data_save_name, path=data_path):
    save_file = path/data_save_name
    if save_file.exists():
        data = load_data(path, data_save_name)
    else:
        save_file.parent.mkdir(exist_ok=True, parents=True)
        vocab = MusicVocab.create()
        processors = [OpenNPFileProcessor(), MusicItemProcessor()]

        data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
        data.save(data_save_name)
    return data
numpy_files = get_files(numpy_path, extensions='.npy', recurse=False); len(numpy_files)

Create final training dataset file (training-data-piano-no-dynamics.pkl)

In [ ]:
all_data = create_databunch(numpy_files, data_save_name=data_save_name); all_data

# TRAIN, SAVE, AND PLOT MODEL PERFOMANCE

Initialize the model

In [ ]:
!nvidia-smi
batch_size = 32
encode_position = True
config = default_config()
dl_tfms = [batch_position_tfm] if encode_position else []
data = load_data(data_path, data_save_name, bs=batch_size, encode_position=encode_position, transpose_range=[0, 12], dl_tfms=dl_tfms)
learn = music_model_learner(data, config=config.copy(), metrics=[accuracy, error_rate])
!nvidia-smi


(Optional) Load Existing Model Checkpoint

In [ ]:
config = default_config()
pretrained_path = Path('/content/musicautobot/SuperPianoModel.pth')
learn = music_model_learner(data, config=config, pretrained_path=pretrained_path, metrics=[accuracy, error_rate])

Train

In [ ]:
learn.fit_one_cycle(100)

Save and Plot

In [ ]:
learn.save('/content/musicautobot/SuperPianoModel')
learn.recorder.plot_lr(show_moms=True)

# GENERATE MUSIC WITH THE MODEL

## SETUP VARIABLES AND LOAD/RE-LOAD THE MODEL (just in case)

In [ ]:
# Location of your midi files
midi_path = Path('/content/musicautobot')

# Location of saved datset
data_path = Path('/content/musicautobot')

# Data and Vocab variables initialization
data = MusicDataBunch.empty(data_path) #for pretrained models
vocab = data.vocab

# Location of pretrained model
pretrained_path = Path('/content/musicautobot/SuperPianoModel.pth')
pretrained_path.parent.mkdir(parents=True, exist_ok=True)

# Initialize the model
config = default_config()
learn = music_model_learner(data, config=config, pretrained_path=pretrained_path)

### PREDICTION

### Choose existing midi file as a starting point

In [ ]:
uploaded = files.upload()
midi_files = get_files(midi_path, recurse=False, extensions='.mid'); midi_files[:5]

In [ ]:
idx = 0
f = midi_files[idx]; f

## Trim (use seed from a MIDI file to predict next sequence/continuation)



In [ ]:
cutoff_beat = 10
item = MusicItem.from_file(f, data.vocab)
seed_item = item.trim_to_beat(cutoff_beat)
#seed_item.show()
play_wav(seed_item.stream)

## GENERATE (CHOOSE ONE OPTION BELOW)

### GENERATE w/starting sequence

Default prediction

In [ ]:
#@title
pred, full = learn.predict(seed_item, n_words=512)
#pred.show()
pred.stream.write('midi', '/content/musicautobot/output.mid')
play_wav(pred.stream)

Custom prediction

In [ ]:
#@title
pred, full = learn.predict(seed_item, n_words=512, temperatures=(1.4,0.6), min_bars=12, top_k=40, top_p=0.2)
#pred.show()
pred.stream.write('midi', '/content/musicautobot/output.mid')
play_wav(pred.stream)

Add above segment to seed segment

In [ ]:
#@title
seed_item = seed_item.append(pred)
#seed_item.show()
seed_item.stream.write('midi', '/content/musicautobot/output.mid')
play_wav(seed_item.stream)

### GENERATE w/o a starting sequence

In [ ]:
empty_item = MusicItem.empty(vocab)
pred, full = learn.predict(empty_item, n_words=256)
#pred.show()
pred.stream.write('midi', '/content/musicautobot/output.mid')
play_wav(pred.stream)